In [1]:
%matplotlib auto
import numpy as np
import matplotlib.pyplot as plt
import csv

def add_noise(signal, snr_db):
    """add noise to trace based on SNR

    Args:
        signal (np.array): original signal without noise
        snr_db (float64): SNR

    Returns:
        np.array: signal with noise added
    """
    signal_power = np.mean(np.abs(signal) ** 2)
    snr = 10 ** (snr_db / 10)
    noise_power = signal_power / snr
    noise = np.sqrt(noise_power) * np.random.randn(*signal.shape)
    noisy_signal = signal + noise 
    return noisy_signal

def prob(n, cov_mat, N):
    """compute probability of tamplate

    Args:
        n (np.array): noise vector for target trace and tamplate
        cov_mat (np.array): covariance matrix of tamplate
        N (int): number of POI

    Returns:
        float64: the probability of tamplate
    """
    det = np.linalg.det(cov_mat)
    cov_mat_inv = np.linalg.pinv(cov_mat)
    return 1/np.sqrt(det * pow(2*np.pi,N))*np.exp(-0.5*np.dot(np.dot(n.T,cov_mat_inv), n))

def tamplate_build(POI_lst:list, profiled_trc:np.array, SNR):
    """build tamplates

    Args:
        POI_lst (list): list of POI
        profiled_trc (np.array): 2-D array of profiled traces
        SNR (float64): SNR

    Returns:
        list: tamplates including mean trc and cov mat
    """
    POI_num = len(POI_lst)
    tamplate_num = len(profiled_trc)
    profiled_trc_num = len(profiled_trc[0])
    profiled_trc_POI = np.zeros([tamplate_num, profiled_trc_num, POI_num])
    for i in range(0, tamplate_num):
        for j in range(0, profiled_trc_num):
            trc = add_noise(profiled_trc[i][j], SNR)
            for k in range(0, POI_num):
                profiled_trc_POI[i][j][k] = trc[POI_lst[k]]

    # build tamplates
    # compute average trace Mi for each tamplate
    profiled_trc_POI_mean = np.zeros([tamplate_num, POI_num])
    for i in range(0, tamplate_num):
        profiled_trc_POI_mean[i] = np.mean(profiled_trc_POI[i], axis=0)

    # compute noise vector for all L sample traces
    noise_vec = np.zeros([tamplate_num, profiled_trc_num, POI_num])
    for i in range(0, tamplate_num):
        for j in range(0, profiled_trc_num):
            for k in range(0, POI_num):
                noise_vec[i][j][k] =  profiled_trc_POI[i][j][k] - profiled_trc_POI_mean[i][k]

    # compute cov matrix of corresponding noise vector
    cov_mat = np.zeros([tamplate_num, POI_num, POI_num])
    for i in range(0, tamplate_num):
        cov_mat[i] = np.cov(noise_vec[i], rowvar=False)
    
    return [profiled_trc_POI_mean, cov_mat]

def tamplate_match(POI_lst:list, trc:np.array, hw_tamplate:list, SNR):
    """tamplate matching

    Args:
        POI_lst (list): list of POI
        trc (np.array): 2-D array of target traces
        hw_tamplate (list): tamplates including mean trc and cov mat
        SNR (float64): SNR

    Returns:
        list: the matching results of hanmming weight and probability
    """
    profiled_trc_mean, cov_mat = hw_tamplate
    POI_num = len(POI_lst)
    test_trc_num = len(trc)
    test_trc = np.zeros([test_trc_num, POI_num])
    for i in range(0, test_trc_num):
        trc_nv = add_noise(trc[i], SNR)
        for k in range(0, POI_num):
            test_trc[i][k] = trc_nv[POI_lst[k]]

    # compute the probability of test traces
    tamplate_num = len(profiled_trc_mean)
    poss_value_lst = np.zeros([test_trc_num, tamplate_num])
    for k in range(0, test_trc_num):
        nv = np.zeros([tamplate_num, POI_num])
        for i in range(0, tamplate_num):
            for j in range(0, POI_num):
                nv[i][j] = test_trc[k][j] - profiled_trc_mean[i][j]
            poss_value_lst[k][i] = prob(nv[i], cov_mat[i], POI_num)

    # sort Hamming weights according to probability
    predict_hw_lst = np.zeros([test_trc_num, tamplate_num])
    predict_prob_lst = np.zeros([test_trc_num, tamplate_num])
    for k in range(0, test_trc_num):
        val = poss_value_lst[k]
        hw = np.array(list(range(0, tamplate_num))) 
        index = np.argsort(val)
        predict_hw_lst[k] = hw[index]
        predict_prob_lst[k] = val[index]
    
    return [predict_hw_lst, predict_prob_lst]

def coef_search_alg(c, predict_ldr_res, predict_str_res):
    """search algorithm to recover the final coeffs

    Args:
        cons_num (int): number of constraints
        c (list): list of c values
        predict_ldr_res (list): matching reasults of ldr (hamming weights and probability)
        predict_str_res (list): matching reasults of str (hamming weights and probability)

    Returns:
        list: final list of s value
        list: final list of probability  
    """
    predict_hw_ldr_lst, predict_prob_ldr_lst = predict_ldr_res
    predict_hw_str_lst, predict_prob_str_lst = predict_str_res
    # search alg.
    Q = 8380417
    cons_num = len(c)
    hw_ldr_lst = [int(i) for i in predict_hw_ldr_lst[0].tolist()]
    hw_ldr_lst = hw_ldr_lst[-6:]
    prob_ldr_lst = predict_prob_ldr_lst[0].tolist()
    prob_ldr_lst = prob_ldr_lst[-6:]

    hw_str_lst = []
    prob_str_lst = []
    for idc in range(0, cons_num):
        v = [int(i) for i in predict_hw_str_lst[idc].tolist()]
        v = v[-5:]
        pv = predict_prob_str_lst[idc].tolist()
        pv = pv[-5:]
        hw_str_lst.append(v)
        prob_str_lst.append(pv)

    final_s_lst = []
    final_p_lst = []
    print('search begin')
    counter = 0
    for s in range(-2-4*Q, 2+4*Q+1):
        counter+=1
        if counter % 1000000 == 0:
            print(f'{counter/(4+8*Q)*100}%')
        hws = HW(s)
        if hws not in hw_ldr_lst:
            continue
        ids = hw_ldr_lst.index(hws)
        p = prob_ldr_lst[ids]
        for idc in range(0, cons_num):
            hwx = HW(montgomery_reduce(c[idc]*s))
            if hwx not in hw_str_lst[idc]:
                p = 0
                break
            idx = hw_str_lst[idc].index(hwx)
            p *= prob_str_lst[idc][idx]
        if p != 0:
            final_s_lst.append(s) 
            final_p_lst.append(p)
        
    print('search end')
    s_lst = np.array(final_s_lst) 
    p_lst = np.array(final_p_lst)
    index = np.argsort(p_lst)
    s_lst = s_lst[index]
    p_lst = p_lst[index]

    return s_lst, p_lst  


Using matplotlib backend: TkAgg


In [16]:
trc_len = 421
POI_lst = [36, 38]
tamplate_num = 33
profiled_trc_num = 500
Kase = 10
test_trc_num = 1000
SNR = 100


profiled_trc = np.zeros([tamplate_num, profiled_trc_num, trc_len])
for i in range(0, tamplate_num):
    for j in range(0, profiled_trc_num):
        profiled_trc[i][j] = np.loadtxt(f"ldr_profile/hw{i}/{j+1}.trc")
tamplate_ldr = tamplate_build(POI_lst, profiled_trc, SNR)


ave_HW1 = 0
ave_HW6 = 0
f = open("ldr_test/rnd_ldr_test_hw.txt", "r") # read correct hw of every case
flines = f.readlines()
for kase in range(0, Kase):
    test_trc_ldr = np.zeros([test_trc_num, trc_len])
    for i in range(0, test_trc_num):
        test_trc_ldr[i] = np.loadtxt(f"ldr_test/Case{kase}/{i+1}.trc")
    [predict_hw_lst, predict_prob_lst] = tamplate_match(POI_lst, test_trc_ldr, tamplate_ldr, SNR)
    
    # hw answer of the case
    ans = flines[2*kase+1].split(",")
    ans = [int(i) for i in ans[0:-1]]
    # compute the accuracy of tamplates
    HW1_hit = 0
    HW6_hit = 0
    for k in range(0, test_trc_num):
        if predict_hw_lst[k][-1] == ans[k]:
            HW1_hit += 1
            ave_HW1 += 1
        if ans[k] in predict_hw_lst[k][-6:]:
            HW6_hit += 1
            ave_HW6 += 1
    print(f'Case{kase}: HW1_rate = {HW1_hit/test_trc_num} HW6_rate = {HW6_hit/test_trc_num}')
print(f"average: HW1_rate = {ave_HW1/(test_trc_num*Kase)} HW6_rate = {ave_HW6/(test_trc_num*Kase)}")

Case0: HW1_rate = 0.277 HW6_rate = 0.977
Case1: HW1_rate = 0.268 HW6_rate = 0.979
Case2: HW1_rate = 0.284 HW6_rate = 0.983
Case3: HW1_rate = 0.264 HW6_rate = 0.978
Case4: HW1_rate = 0.265 HW6_rate = 0.982
Case5: HW1_rate = 0.262 HW6_rate = 0.988
Case6: HW1_rate = 0.247 HW6_rate = 0.976
Case7: HW1_rate = 0.283 HW6_rate = 0.979
Case8: HW1_rate = 0.262 HW6_rate = 0.978
Case9: HW1_rate = 0.258 HW6_rate = 0.975
average: HW1_rate = 0.267 HW6_rate = 0.9795


In [5]:
# test tamplate accuracy of each tamplate
# res = [['HW1_hit', 'HW5_hit']]
trc_len = 421
ave_HW1 = 0
ave_HW5 = 0

# load test traces from file
for test_hw in range(0, 33):
    test_trc_num = 1000
    test_trc_ldr = np.zeros([test_trc_num, trc_len])
    for i in range(0, test_trc_num):
        test_trc_ldr[i] = np.loadtxt(f"hw_ldr_test/hw{test_hw}/{i+1}.trc")

    [predict_hw_lst, predict_prob_lst] = tamplate_match(POI_lst, test_trc_ldr, tamplate_ldr, SNR)

    # compute the accuracy of tamplates
    HW1_hit = 0
    HW5_hit = 0
    for k in range(0, test_trc_num):
        if predict_hw_lst[k][-1] == test_hw:
            HW1_hit += 1
            ave_HW1 += 1
        if test_hw in predict_hw_lst[k][-5:]:
            HW5_hit += 1
            ave_HW5 += 1 
    print(f'HW{test_hw}: HW1_rate = {HW1_hit/test_trc_num} HW5_rate = {HW5_hit/test_trc_num}')
    # res.append([test_hw, HW1_hit/test_trc_num, HW5_hit/test_trc_num])
print(f"average: HW1_rate = {ave_HW1/(test_trc_num*33)} HW5_rate = {ave_HW5/(test_trc_num*33)}")
'''
csvfile = open("hw_ldr_rate.csv", 'w', newline='')
csvwriter = csv.writer(csvfile)
for row in res:
    csvwriter.writerow(row)
csvfile.close()
'''


HW0: HW1_rate = 1.0 HW5_rate = 1.0
HW1: HW1_rate = 0.943 HW5_rate = 1.0
HW2: HW1_rate = 0.697 HW5_rate = 1.0
HW3: HW1_rate = 0.599 HW5_rate = 1.0
HW4: HW1_rate = 0.495 HW5_rate = 1.0
HW5: HW1_rate = 0.399 HW5_rate = 1.0
HW6: HW1_rate = 0.427 HW5_rate = 1.0
HW7: HW1_rate = 0.35 HW5_rate = 0.998
HW8: HW1_rate = 0.342 HW5_rate = 0.998
HW9: HW1_rate = 0.311 HW5_rate = 0.996
HW10: HW1_rate = 0.349 HW5_rate = 0.992
HW11: HW1_rate = 0.297 HW5_rate = 0.99
HW12: HW1_rate = 0.348 HW5_rate = 0.985
HW13: HW1_rate = 0.253 HW5_rate = 0.985
HW14: HW1_rate = 0.344 HW5_rate = 0.988
HW15: HW1_rate = 0.328 HW5_rate = 0.982
HW16: HW1_rate = 0.378 HW5_rate = 0.973
HW17: HW1_rate = 0.325 HW5_rate = 0.984
HW18: HW1_rate = 0.338 HW5_rate = 0.973
HW19: HW1_rate = 0.308 HW5_rate = 0.987
HW20: HW1_rate = 0.342 HW5_rate = 0.985
HW21: HW1_rate = 0.358 HW5_rate = 0.994
HW22: HW1_rate = 0.304 HW5_rate = 0.994
HW23: HW1_rate = 0.341 HW5_rate = 0.995
HW24: HW1_rate = 0.349 HW5_rate = 0.997
HW25: HW1_rate = 0.403 HW5_r

'\ncsvfile = open("hw_ldr_rate.csv", \'w\', newline=\'\')\ncsvwriter = csv.writer(csvfile)\nfor row in res:\n    csvwriter.writerow(row)\ncsvfile.close()\n'

In [3]:
trc_len = 421
tamplate_num = 33
POI_lst = [174, 179, 181, 203]
profiled_trc_num = 500
Kase = 10
test_trc_num = 1000
SNR = 100

profiled_trc = np.zeros([tamplate_num, profiled_trc_num, trc_len])
for i in range(0, tamplate_num):
    for j in range(0, profiled_trc_num):
        profiled_trc[i][j] = np.loadtxt(f"str_profile/hw{i}/{j+1}.trc")
tamplate_str = tamplate_build(POI_lst, profiled_trc, SNR)

ave_HW1 = 0
ave_HW6 = 0
f = open("str_test/rnd_str_test_hw.txt", "r") # read correct hw of each case
flines = f.readlines()
for kase in range(0, Kase):
    test_trc_str = np.zeros([test_trc_num, trc_len])
    for i in range(0, test_trc_num):
        test_trc_str[i] = np.loadtxt(f"str_test/Case{kase}/{i+1}.trc")
    [predict_hw_lst, predict_prob_lst] = tamplate_match(POI_lst, test_trc_str, tamplate_str, SNR)
    
    # hw answer of the case
    ans = flines[2*kase+1].split(",")
    ans = [int(i) for i in ans[0:-1]]

    # compute the accuracy of tamplates
    HW1_hit = 0
    HW6_hit = 0
    for k in range(0, test_trc_num):
        if predict_hw_lst[k][-1] == ans[k]:
            HW1_hit += 1
            ave_HW1 += 1
        if ans[k] in predict_hw_lst[k][-6:]:
            HW6_hit += 1
            ave_HW6 += 1
    print(f'Case{kase}: HW1_rate = {HW1_hit/test_trc_num} HW6_rate = {HW6_hit/test_trc_num}')
print(f"average: HW1_rate = {ave_HW1/(test_trc_num*Kase)} HW6_rate = {ave_HW6/(test_trc_num*Kase)}")   

Case0: HW1_rate = 0.476 HW6_rate = 0.998
Case1: HW1_rate = 0.485 HW6_rate = 0.997
Case2: HW1_rate = 0.443 HW6_rate = 0.998
Case3: HW1_rate = 0.464 HW6_rate = 0.998
Case4: HW1_rate = 0.485 HW6_rate = 0.999
Case5: HW1_rate = 0.476 HW6_rate = 0.999
Case6: HW1_rate = 0.478 HW6_rate = 0.997
Case7: HW1_rate = 0.459 HW6_rate = 0.999
Case8: HW1_rate = 0.482 HW6_rate = 0.998
Case9: HW1_rate = 0.467 HW6_rate = 0.999
average: HW1_rate = 0.4715 HW6_rate = 0.9982


In [8]:
# test tamplate accuracy of each tamplate
# res = [['HW1_hit', 'HW5_hit']]
trc_len = 421
ave_HW1 = 0
ave_HW5 = 0

# load test traces from file
for test_hw in range(0, 33):
    test_trc_num = 1000
    test_trc_str = np.zeros([test_trc_num, trc_len])
    for i in range(0, test_trc_num):
        test_trc_str[i] = np.loadtxt(f"hw_str_test/hw{test_hw}/{i+1}.trc")

    [predict_hw_lst, predict_prob_lst] = tamplate_match(POI_lst, test_trc_str, tamplate_str, SNR)

    # compute the accuracy of tamplates
    HW1_hit = 0
    HW5_hit = 0
    for k in range(0, test_trc_num):
        if predict_hw_lst[k][-1] == test_hw:
            HW1_hit += 1
            ave_HW1 += 1
        if test_hw in predict_hw_lst[k][-5:]:
            HW5_hit += 1
            ave_HW5 += 1 
    print(f'HW{test_hw}: HW1_rate = {HW1_hit/test_trc_num} HW5_rate = {HW5_hit/test_trc_num}')
    # res.append([test_hw, HW1_hit/test_trc_num, HW5_hit/test_trc_num])
print(f"average: HW1_rate = {ave_HW1/(test_trc_num*33)} HW5_rate = {ave_HW5/(test_trc_num*33)}")
'''
csvfile = open("hw_ldr_rate.csv", 'w', newline='')
csvwriter = csv.writer(csvfile)
for row in res:
    csvwriter.writerow(row)
csvfile.close()
'''


HW0: HW1_rate = 0.979 HW5_rate = 1.0
HW1: HW1_rate = 0.694 HW5_rate = 1.0
HW2: HW1_rate = 0.525 HW5_rate = 0.978
HW3: HW1_rate = 0.577 HW5_rate = 0.998
HW4: HW1_rate = 0.558 HW5_rate = 0.999
HW5: HW1_rate = 0.595 HW5_rate = 1.0
HW6: HW1_rate = 0.571 HW5_rate = 0.999
HW7: HW1_rate = 0.557 HW5_rate = 1.0
HW8: HW1_rate = 0.579 HW5_rate = 1.0
HW9: HW1_rate = 0.586 HW5_rate = 1.0
HW10: HW1_rate = 0.526 HW5_rate = 1.0
HW11: HW1_rate = 0.567 HW5_rate = 1.0
HW12: HW1_rate = 0.54 HW5_rate = 1.0
HW13: HW1_rate = 0.615 HW5_rate = 0.997
HW14: HW1_rate = 0.498 HW5_rate = 0.989
HW15: HW1_rate = 0.399 HW5_rate = 1.0
HW16: HW1_rate = 0.447 HW5_rate = 0.998
HW17: HW1_rate = 0.329 HW5_rate = 0.998
HW18: HW1_rate = 0.564 HW5_rate = 0.988
HW19: HW1_rate = 0.449 HW5_rate = 0.99
HW20: HW1_rate = 0.379 HW5_rate = 0.994
HW21: HW1_rate = 0.386 HW5_rate = 0.978
HW22: HW1_rate = 0.354 HW5_rate = 0.981
HW23: HW1_rate = 0.42 HW5_rate = 0.986
HW24: HW1_rate = 0.381 HW5_rate = 0.988
HW25: HW1_rate = 0.41 HW5_rate = 

'\ncsvfile = open("hw_ldr_rate.csv", \'w\', newline=\'\')\ncsvwriter = csv.writer(csvfile)\nfor row in res:\n    csvwriter.writerow(row)\ncsvfile.close()\n'

In [10]:
def HW(a):
    count = 0
    a = a & 0xFFFFFFFF
    while a:
        a &= a - 1
        count += 1
    return count
def montgomery_reduce(a):
    r = (a * 8265825) % 8380417
    if r > (8380417 >> 1):
        r -= 8380417
    return r

In [20]:
# load test traces from cons_ldr(including 16 constraints for s = 16384)
c = [172469,-4242129,3584314,-4283808,1849066,3454452,-3603994,-601242,-3209795,2722127,-5616586,-1040334,-1241650,6766294,-5769523,283639,]
s = 16384
POI_lst = [36, 38] 
test_trc_num = 16
trc_len = 421
test_trc_ldr = np.zeros([test_trc_num, trc_len])
for i in range(0, test_trc_num):
    test_trc_ldr[i] = np.loadtxt(f"cons_ldr/trc{i+1}.trc")

[predict_hw_ldr_lst, predict_prob_ldr_lst] = tamplate_match(POI_lst,test_trc_ldr, tamplate_ldr, SNR)

for k in range(0, test_trc_num):
    print(f'{HW(s)} {predict_hw_ldr_lst[k][-6:]} ')


1 [6. 5. 4. 3. 2. 1.] 
1 [6. 5. 4. 3. 2. 1.] 
1 [6. 5. 4. 3. 2. 1.] 
1 [6. 5. 4. 3. 2. 1.] 
1 [6. 5. 4. 3. 2. 1.] 
1 [6. 5. 4. 3. 2. 1.] 
1 [6. 5. 4. 3. 2. 1.] 
1 [6. 5. 4. 3. 2. 1.] 
1 [6. 5. 4. 3. 2. 1.] 
1 [6. 5. 4. 3. 2. 1.] 
1 [6. 5. 4. 3. 2. 1.] 
1 [6. 5. 4. 3. 2. 1.] 
1 [6. 5. 4. 3. 2. 1.] 
1 [6. 5. 4. 3. 2. 1.] 
1 [6. 5. 4. 3. 2. 1.] 
1 [6. 5. 4. 3. 2. 1.] 


In [23]:
c = [172469,-4242129,3584314,-4283808,1849066,3454452,-3603994,-601242,-3209795,2722127,-5616586,-1040334,-1241650,6766294,-5769523,283639,]
s = 16384
POI_lst = [174, 179, 181, 203] 
test_trc_num = 16
trc_len = 421
test_trc_str = np.zeros([test_trc_num, trc_len])
for i in range(0, test_trc_num):
    test_trc_str[i] = np.loadtxt(f"cons_ldr/trc{i+1}.trc")

[predict_hw_str_lst, predict_prob_str_lst] = tamplate_match(POI_lst,test_trc_str, tamplate_str, SNR)

for k in range(0, test_trc_num):
    print(f'{HW(montgomery_reduce(c[k]*s))} {predict_hw_str_lst[k][-5:]}')

24 [24. 28. 25. 27. 26.]
22 [20. 21. 24. 22. 23.]
20 [19. 23. 20. 22. 21.]
10 [14. 10. 13. 12. 11.]
22 [25. 21. 24. 22. 23.]
13 [16. 12. 15. 13. 14.]
19 [18. 22. 21. 19. 20.]
13 [16. 12. 15. 13. 14.]
11 [15. 11. 14. 13. 12.]
11 [14. 10. 13. 12. 11.]
19 [18. 22. 21. 19. 20.]
15 [13. 14. 15. 17. 16.]
11 [10. 14. 13. 11. 12.]
16 [19. 15. 18. 16. 17.]
17 [15. 19. 16. 17. 18.]
11 [11. 15. 12. 14. 13.]


In [25]:
c = [172469,-4242129,3584314,-4283808,1849066,3454452,-3603994,-601242,-3209795,2722127,-5616586,-1040334,-1241650,6766294,-5769523,283639,]
c = [172469,-4242129,3584314,-4283808,1849066,3454452,-3603994,-601242,-3209795,2722127]
test_trc = np.zeros([16, 421])
for i in range(0, 16):
    test_trc[i] = np.loadtxt(f"cons_ldr/trc{i+1}.trc")
predict_ldr_res = tamplate_match([36,38], test_trc, tamplate_ldr, 100)
predict_str_res = tamplate_match([174, 179, 181, 203], test_trc, tamplate_str, 100)
[s_lst, p_lst] = coef_search_alg(c, predict_ldr_res, predict_str_res)

print(s_lst)

search begin
1.4915724664075507%
2.9831449328151014%
4.474717399222652%
5.966289865630203%
7.457862332037754%
8.949434798445305%
10.441007264852855%
11.932579731260406%
13.424152197667958%
14.915724664075508%
16.40729713048306%
17.89886959689061%
19.390442063298156%
20.88201452970571%
22.37358699611326%
23.86515946252081%
25.35673192892836%
26.848304395335916%
28.339876861743463%
29.831449328151017%
31.323021794558564%
32.81459426096612%
34.306166727373665%
35.79773919378122%
37.289311660188766%
38.78088412659631%
40.27245659300387%
41.76402905941142%
43.25560152581897%
44.74717399222652%
46.238746458634076%
47.73031892504162%
49.22189139144918%
50.71346385785672%
52.20503632426428%
53.69660879067183%
55.18818125707937%
56.679753723486925%
58.17132618989448%
59.66289865630203%
61.15447112270957%
62.64604358911713%
64.13761605552469%
65.62918852193224%
67.12076098833978%
68.61233345474733%
70.10390592115489%
71.59547838756244%
73.08705085396998%
74.57862332037753%
76.07019578678509%
77.

In [101]:
c = [172469,-4242129,3584314,-4283808,1849066,3454452,-3603994,-601242,-3209795,2722127,-5616586,-1040334,-1241650,6766294,-5769523,283639,]
s = [3553832,-3536216,5534040,2789784,11439920,4285088,2634103,7792489,1284349,-1497527,-4663214,-53276,-5897019,-6943473,-7417610,297642,-4652241,1784083,4097018,2779188,-1471660,-4326046,1090656,1167482,12182172,6852818,6820014,4962340,14052006,7764244,7679549,3469097,-1517282,1734462,2681957,-3133045,11809228,12244222,3569160,5648146,77071,-7682303,-4627768,1270240,1654452,1299638,4745430,4236744,9335601,1449485,8351037,16170129,5552261,8896631,-1822685,6330957,8676700,15684766,9603032,2169698,14730136,11578436,7746611,8420181,-8008781,-2476689,-3560371,-11746815,-6843795,-1683477,-7727036,-10367076,-5006941,-10476567,-14018304,-7815060,-8184557,-13205923,-17895346,-12454094,-7339143,-4612715,-604676,-2968990,8675850,4501796,1953435,-4065413,-4523995,-9190577,-2861752,5417604,-11239923,-12264485,-10484832,-4518024,2136701,8858793,-2799330,4295116,5138506,1197154,2167591,4233453,8817008,1450626,1266157,250437,-3216863,-672681,6469298,4245266,-2186932,-1303020,8409279,2714349,-1862341,-7199989,-2841439,-6814939,-8030195,-6163969,-11786247,-4028265,-11123440,-11299674,-9925077,-15535365,-3484302,-1772778,1452334,5917302,7232428,2033186,11117276,6329706,-1320715,-7349143,-266470,-525160,-11199167,-9063217,-2120257,-4772159,3269155,109349,6112987,1348333,3861162,-1452678,2508953,9464003,5332459,8101267,2932580,1616194,-3413450,3777154,8600064,886580,5837885,-2530955,9810833,5293961,72846,3958306,3391312,-263940,-1709708,2280268,4695757,1948603,408846,5240428,-3528254,2268916,2535736,9450608,10421698,17661366,8982289,2245463,-3401262,944030,8295606,14647640,12592019,8586339,20611038,16483974,15235716,13268636,-3324160,-3317286,105525,5501613,-3791138,4474510,-5147178,-2974566,-7859931,-14407183,-11434443,-6968579,-1206722,-7653730,-7168845,389425,-6616321,-14800975,-8997530,-9079750,-11480128,-10706196,-15890514,-10881362,-1202134,-2227914,-11855845,-4407003,-729871,-8808569,-12220875,-5856389,2139413,5846591,2588570,-1171334,11735719,5837035,4048090,-554468,-4092806,1982718,-8347349,-4315019,4464507,5842979,-1194100,6785838,-7478359,-12649441,-10354832,-6871956,-2595548,-7743328,-3504648,-7548408,-3995448,3439764,-10371883,-4516133,-5039613,-1768753,2773526,6174020]
cons_num = 10
c = c[:cons_num]
s_recover = []
p_recover = []
for idi in range(0, 256):
    print(f"recover coef no.{idi}")
    test_trc = np.zeros([17, 421])
    for i in range(1, 17):
        trc = np.loadtxt(f"cons/{i+1+17*idi}.trc")
        if len(trc)==425:
            trc = trc[0:421]
        test_trc[i] = trc
    predict_ldr_res = tamplate_match([36,38], test_trc, tamplate_ldr, 100)
    predict_str_res = tamplate_match([174, 179, 181, 203], test_trc, tamplate_str, 100)
    [s_lst, p_lst] = coef_search_alg(c, predict_ldr_res, predict_str_res)
    if len(s_lst) == 0:
        s_recover.append(0)
    else:
        s_recover.append(s_lst[-1])
        p_recover.append(p_lst[-1])
sr = 0
for i in range(0, 256):
    if s_recover[i] == s[i]:
        sr+=1
print(f'SR = {sr/256}')

recover coef no.0
search begin
1.4915724664075507%
2.9831449328151014%
4.474717399222652%
5.966289865630203%
7.457862332037754%
8.949434798445305%
10.441007264852855%
11.932579731260406%
13.424152197667958%
14.915724664075508%
16.40729713048306%
17.89886959689061%
19.390442063298156%
20.88201452970571%
22.37358699611326%
23.86515946252081%
25.35673192892836%
26.848304395335916%
28.339876861743463%
29.831449328151017%
31.323021794558564%
32.81459426096612%
34.306166727373665%
35.79773919378122%
37.289311660188766%
38.78088412659631%
40.27245659300387%
41.76402905941142%
43.25560152581897%
44.74717399222652%
46.238746458634076%
47.73031892504162%
49.22189139144918%
50.71346385785672%
52.20503632426428%
53.69660879067183%
55.18818125707937%
56.679753723486925%
58.17132618989448%
59.66289865630203%
61.15447112270957%
62.64604358911713%
64.13761605552469%
65.62918852193224%
67.12076098833978%
68.61233345474733%
70.10390592115489%
71.59547838756244%
73.08705085396998%
74.57862332037753%
76.0

KeyboardInterrupt: 